# Парсинг сайта автодиспетчер

In [1]:
import pandas as pd
import numpy as np
import math
import os
import requests
from bs4 import BeautifulSoup
import time
import datetime

# Suitable default display for floats
pd.options.display.float_format = '{:,.3f}'.format

# Отобразить все колонки
pd.set_option('max_columns', None)


In [2]:
# Открываем файл
df = pd.read_excel(r"C:\Users\Александр\OneDrive\Python\Python projects\015_Дистанция по геолокации\2023 08 22_Геоданные для расчета.xlsx")

In [17]:
# Подготовка df
df1 = df[['Заявка, id', 'МНО, Адрес', 'МНО_Координаты (Ш,Д)', 'Объект переработки, Адрес', 'ОП_Координаты (Ш,Д)']].copy()

# Переименование колонок
df1.rename(columns={'Заявка, id': 'bid_id', 'МНО, Адрес': 'mno_add', 'МНО_Координаты (Ш,Д)': 'mno_geo', 
                     'Объект переработки, Адрес': 'orec_add', 'ОП_Координаты (Ш,Д)': 'orec_geo'}, inplace=True)

# Удаляем адреса без конечной геолокации
df1.dropna(subset=['orec_add'], inplace=True)
df1 = df1.reset_index(drop=True)
len(df1)

636

In [18]:
# # Отладка
# df1 = df1.query('bid_id <= 19365')
# df1 = df1.reset_index(drop=True)
# len(df1['bid_id'])

In [19]:
# # Костыль для прогонки кода:
# df1 = df1.loc[df1.index >= 500]
# df1

In [20]:
# # Сохранение:
# def save_file(df1, filename):
#     with pd.ExcelWriter(filename) as writer:  
#         df1.to_excel(writer)

# # Парсер дистанции
# def get_distance(from_address, to_address):
#     try:
#         url = f'https://www.avtodispetcher.ru/distance/?from={from_address}&to={to_address}'
#         resp = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
#         soup = BeautifulSoup(resp.content, 'html.parser')
#         distance_elem = soup.find('span', {'id': 'totalDistance'})
#         time_elem = soup.find('span', {'id': 'totalTime'})
#         distance = distance_elem.text if distance_elem else '0'
#         hours = time_elem.text.strip() if time_elem else '0'
#         hours = ' '.join(hours.split()) # удаляем лишние пробелы
#         time.sleep(0.5)
#     except:
#         distance = 0
#         hours = 0
#     return distance, time

# df1[['distance', 'time']] = df1.apply(lambda x: pd.Series(get_distance(x['mno_geo'], x['orec_geo'])), axis=1)

# # Преобразование дистанции
# df1['distance'] = df1['distance'].astype('int64')

# # Преобразование времени
# df1['hour'] = np.where(df1['time'].str.contains('ч'), df1['time'].str.extract(r'(\d+)\s*ч\s*(\d+)\s*мин', expand=True)[0].fillna(0).astype('int64'), 0)
# df1['minute'] = np.where(df1['time'].str.contains('ч'), df1['time'].str.extract(r'(\d+)\s*ч\s*(\d+)\s*мин', expand=True)[1].fillna(0).astype('int64'), df1['time'].str.extract(r'(\d+)\s*мин', expand=True)[0].fillna(0).astype('int64'))


In [21]:
# # Сохранение:
# def save_file(df1, filepath):
#     folder_path, filename = os.path.split(filepath)
#     with pd.ExcelWriter(filepath) as writer:  
#         df1.to_excel(writer)
        
# # Парсер дистанции
# def get_distance(from_address, to_address):
#     try:
#         url = f'https://www.avtodispetcher.ru/distance/?from={from_address}&to={to_address}'
#         resp = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
#         soup = BeautifulSoup(resp.content, 'html.parser')
#         distance_elem = soup.find('span', {'id': 'totalDistance'})
#         hours_elem = soup.find('span', {'id': 'totalTime'})
#         distance = distance_elem.text if distance_elem else '0'
#         hours = hours_elem.text.strip() if hours_elem else '0'
#         hours = ' '.join(hours.split()) # удаляем лишние пробелы
#         time.sleep(0.5)
#     except:
#         distance = 0
#         hours = 0
#     return distance, hours

# # Расчет расстояния и сохранение каждые 100 значений
# count = 0
# total_rows = len(df1)
# start_time = time.time()  # Запоминаем время начала расчета

# for index, row in df1.iterrows():
#     distance, hours = get_distance(row['mno_geo'], row['orec_geo'])
#     df1.at[index, 'distance'] = distance
#     df1.at[index, 'hours'] = hours
#     count += 1
#     progress = count / total_rows * 100
#     progress_str = f"Progress: {progress:.2f}%"
#     elapsed_time = time.time() - start_time  # Вычисляем прошедшее время
#     elapsed_hours = int(elapsed_time / 3600)  # Преобразуем время в часы
#     elapsed_minutes = int((elapsed_time % 3600) / 60)  # Преобразуем остаток времени в минуты
#     elapsed_seconds = int(elapsed_time % 60)  # Преобразуем остаток времени в секунды
#     time_str = f"Elapsed Time: {elapsed_hours} hours {elapsed_minutes} minutes {elapsed_seconds} seconds"
#     print(progress_str, time_str, end='\r')

#     if count % 100 == 0 or count == total_rows:
#         df1['distance'] = df1['distance'].astype('int64')
#         save_file(df1, f'C:/Users/Александр/OneDrive/Python/Python projects/015_Дистанция по геолокации/Data/calc_distance.xlsx')

In [22]:
# Сохранение:
def save_file(df1, filepath):
    folder_path, filename = os.path.split(filepath)
    with pd.ExcelWriter(filepath) as writer:  
        df1.to_excel(writer)
        
# Парсер дистанции
def get_distance(from_address, to_address):
    try:
        url = f'https://www.avtodispetcher.ru/distance/?from={from_address}&to={to_address}'
        resp = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
        soup = BeautifulSoup(resp.content, 'html.parser')
        distance_elem = soup.find('span', {'id': 'totalDistance'})
        hours_elem = soup.find('span', {'id': 'totalTime'})
        distance = distance_elem.text if distance_elem else '0'
        hours = hours_elem.text.strip() if hours_elem else '0'
        hours = ' '.join(hours.split()) # удаляем лишние пробелы
        time.sleep(0.5)
    except:
        distance = 0
        hours = 0
    return distance, hours

# Расчет расстояния и сохранение каждые 100 значений
count = 0
total_rows = len(df1)
# start_time = time.time()  # Запоминаем время начала расчета

for index, row in df1.iterrows():
    distance, hours = get_distance(row['mno_geo'], row['orec_geo'])
    df1.at[index, 'distance'] = distance
    df1.at[index, 'hours'] = hours
    count += 1
    
    progress = count / total_rows * 100
    progress_str = f"Progress: {progress:.2f}% Count: {count}/{total_rows}"
#     elapsed_time = time.time() - start_time  # Вычисляем прошедшее время
#     elapsed_hours = int(elapsed_time / 3600)  # Преобразуем время в часы
#     elapsed_minutes = int((elapsed_time % 3600) / 60)  # Преобразуем остаток времени в минуты
#     elapsed_seconds = int(elapsed_time % 60)  # Преобразуем остаток времени в секунды
#     time_str = f"| Elapsed Time: {elapsed_hours} hours {elapsed_minutes} minutes {elapsed_seconds} seconds"
#     print(progress_str, time_str, end='\r')
    print(progress_str, end='\r')
    
    if count % 500 == 0 or count == total_rows:
#         df1['distance'] = df1['distance'].fillna(0).astype('int64')
#         df1['hour'] = np.where(df1['hours'].fillna('0ч 0мин').str.contains('ч'), df1['hours'].str.extract(r'(\d+)\s*ч\s*(\d+)\s*мин', expand=True)[0].astype('int64'), 0)
#         df1['minute'] = np.where(df1['hours'].fillna('0ч 0мин').str.contains('ч'), df1['hours'].str.extract(r'(\d+)\s*ч\s*(\d+)\s*мин', expand=True)[1].fillna(0).astype('int64'), df1['hours'].str.extract(r'(\d+)\s*мин', expand=True)[0].fillna(0).astype('int64'))
        save_file(df1, f'C:/Users/Александр/OneDrive/Python/Python projects/015_Дистанция по геолокации/Data/calc_distance.xlsx')
#     elif count == total_rows:
#         df1['distance'] = df1['distance'].fillna(0).astype('int64')
#         df1['hour'] = np.where(df1['hours'].str.contains('ч'), df1['hours'].str.extract(r'(\d+)\s*ч\s*(\d+)\s*мин', expand=True)[0].fillna(0).astype('int64'), 0)
#         df1['minute'] = np.where(df1['hours'].str.contains('ч'), df1['hours'].str.extract(r'(\d+)\s*ч\s*(\d+)\s*мин', expand=True)[1].fillna(0).astype('int64'), df1['hours'].str.extract(r'(\d+)\s*мин', expand=True)[0].fillna(0).astype('int64'))   
#         save_file(df1, f'C:/Users/Александр/OneDrive/Python/Python projects/015_Дистанция по геолокации/Data/calc_distance.xlsx')


In [23]:
df1

,bid_id,mno_add,mno_geo,orec_add,orec_geo,distance,hours
0,73216,"423600, Российская Федерация, Республика Татар...","55.845874,52.001449","Республика Татарстан, г Казань, ул Складская, ...","55.739616,49.111158",2605,28 ч 29 мин
1,73066,"г Орёл, ул Пионерская, д 8","52.97702,36.059193","Белгородская обл, Шебекинский р-н, село Неклюд...","50.545508,36.916204",326,4 ч 12 мин
2,72885,"Ростовская обл, г Зверево, ул Космонавтов, зд 45","48.045235,40.124706","Ростовская область, Семикаракорский район, в 3...","47.514144,40.896821",411,5 ч 13 мин
3,72833,"Краснодарский край, г Тихорецк, ул Ударников, д 2","45.85529,40.097355","Ростовская область, Семикаракорский район, в 3...","47.514144,40.896821",264,3 ч 8 мин
4,72822,"г Калининград, ул Черняховского, д 26","54.719536,20.507307","г Калининград, ул А.Невского, д 240","54.757254,20.552349",7,5 мин
...,...,...,...,...,...,...,...
631,3024,Оренбургская область Первомайский район,"52.808655,52.511942","Московская обл, г Подольск, ул Вишневая, д 7","55.480072,37.574955",0,0
632,2879,"Красноярский край, г. Норильск-Алыкель автодор...","69.360614,88.088723","г Красноярск, ул Кразовская, д 10Г","56.109802,93.02873",6322,69 ч 1 мин
633,2768,"Красноярский край, г Норильск, ул Богдана Хмел...","69.346197,88.210365","г Красноярск, ул Кразовская, д 10Г","56.109802,93.02873",0,0
634,1857,"Ямало-ненецкий автономный округ, Пуровский рай...","64.458927,75.728987","Пермский край, Пермский р-н, деревня Песьянка,...","57.942724,56.066453",4275,45 ч 57 мин
